# 数据清洗与预处理 {#sec-data-cleaning}

::: {.callout-note}
## 本章概要
- **课时**：2课时（第9周）
- **目标**：掌握数据清洗的核心概念和 Pandas 方法
:::

## 学习目标

完成本章后，你将能够：

1. 识别常见的数据质量问题
2. 使用 Pandas 进行数据清洗
3. 用 AI 生成清洗代码

---

## 现实数据是"脏"的

```{mermaid}
mindmap
  root((Data Quality Issues))
    Missing Values
      Empty Cells
      NA Markers
      Placeholders
    Outliers
      Out of Range
      Input Errors
      Extremes
    Inconsistency
      Format Mismatch
      Unit Difference
      Naming Variance
    Duplicates
      Full Duplicate
      Partial Duplicate
      ID Duplicate
```

### 示例：杂乱的学生数据

In [ ]:
#| eval: true
import pandas as pd
import numpy as np

# 模拟"脏"数据
messy_data = pd.DataFrame({
    '学号': ['001', '002', '003', '003', '004', '005', '006'],
    '姓名': ['张三', '李四', None, '王五', '赵六', '孙七', '周八'],
    '成绩': [85, 150, 78, 78, -5, None, 92],  # 150和-5是异常值
    '班级': ['1班', '一班', '1班', '1班', '1 班', '1班', '01班'],  # 格式不一致
    '入学日期': ['2024-09-01', '2024/9/1', '2024.9.1', '2024-09-01', None, '24-9-1', '2024-09-01']
})

print("原始数据（有多种问题）:")
print(messy_data)

---

## 处理缺失值

### 发现缺失值

In [ ]:
#| eval: true
# 检查缺失值
print("缺失值统计:")
print(messy_data.isnull().sum())

print("\n缺失值占比:")
print(messy_data.isnull().mean() * 100)

### 处理策略

```{mermaid}
flowchart TD
    A["发现缺失值"] --> B{"缺失多少?"}
    B -->|"< 5%"| C["删除或填充"]
    B -->|"5-30%"| D["智能填充"]
    B -->|"> 30%"| E["考虑删除该列"]
    
    C --> C1["删除行: dropna()"]
    C --> C2["填充均值/中位数"]
    
    D --> D1["用同组均值填充"]
    D --> D2["用前后值填充"]
    D --> D3["标记为缺失类别"]
```

In [ ]:
#| eval: true
import pandas as pd
import numpy as np

# 创建示例数据
df = pd.DataFrame({
    '姓名': ['张三', None, '王五', '赵六'],
    '成绩': [85, 90, None, 88],
    '年龄': [20, 21, None, 19]
})

print("原始数据:")
print(df)

# 方法1：删除含缺失值的行
df_dropna = df.dropna()
print("\n删除缺失值后:")
print(df_dropna)

# 方法2：填充固定值
df_fill = df.fillna({'姓名': '未知', '成绩': 0, '年龄': df['年龄'].mean()})
print("\n填充后:")
print(df_fill)

# 方法3：用均值填充数值列
df_mean = df.copy()
df_mean['成绩'] = df_mean['成绩'].fillna(df_mean['成绩'].mean())
df_mean['年龄'] = df_mean['年龄'].fillna(df_mean['年龄'].median())
print("\n均值/中位数填充后:")
print(df_mean)

---

## 处理异常值

### 发现异常值

In [ ]:
#| eval: true
import pandas as pd
import numpy as np

# 示例数据
scores = pd.Series([85, 92, 78, 150, 60, -5, 88, 95, 72, 300])

# 方法1：范围检查
print("超出 0-100 的值:")
print(scores[(scores < 0) | (scores > 100)])

# 方法2：统计方法（3σ原则）
mean = scores.mean()
std = scores.std()
outliers = scores[(scores < mean - 3*std) | (scores > mean + 3*std)]
print(f"\n统计异常值（3σ）: {outliers.tolist()}")

# 方法3：四分位距法（IQR）
Q1 = scores.quantile(0.25)
Q3 = scores.quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
outliers_iqr = scores[(scores < lower) | (scores > upper)]
print(f"IQR异常值: {outliers_iqr.tolist()}")

### 处理策略

In [ ]:
#| eval: true
# 原始数据
scores = pd.Series([85, 92, 78, 150, 60, -5, 88, 95, 72, 300])

# 方法1：删除异常值
valid_scores = scores[(scores >= 0) & (scores <= 100)]
print(f"删除异常值: {valid_scores.tolist()}")

# 方法2：截断（Winsorize）
clipped = scores.clip(lower=0, upper=100)
print(f"截断处理: {clipped.tolist()}")

# 方法3：替换为 NaN，后续单独处理
replaced = scores.where((scores >= 0) & (scores <= 100), np.nan)
print(f"替换为NaN: {replaced.tolist()}")

---

## 统一数据格式

### 日期格式统一

In [ ]:
#| eval: true
import pandas as pd

# 各种日期格式
dates = pd.Series(['2024-09-01', '2024/9/1', '2024.9.1', '24-9-1', 'Sep 1, 2024'])

# 统一转换
try:
    unified = pd.to_datetime(dates, errors='coerce')
    print("统一后的日期:")
    print(unified)
except Exception as e:
    print(f"转换错误: {e}")

### 文本格式统一

In [ ]:
#| eval: true
import pandas as pd

# 不一致的班级名称
classes = pd.Series(['1班', '一班', '1 班', '01班', ' 1班 ', '一 班'])

# 文本清洗函数
def clean_class_name(name):
    """统一班级名称格式"""
    if pd.isna(name):
        return None
    
    # 去除空格
    name = name.strip().replace(' ', '')
    
    # 中文数字转阿拉伯数字
    cn_to_num = {'一': '1', '二': '2', '三': '3', '四': '4', '五': '5'}
    for cn, num in cn_to_num.items():
        name = name.replace(cn, num)
    
    # 去除前导零
    name = name.lstrip('0')
    
    # 确保以"班"结尾
    if not name.endswith('班'):
        name = name + '班'
    
    return name

# 应用清洗
cleaned = classes.apply(clean_class_name)
print("清洗前后对比:")
comparison = pd.DataFrame({'原始': classes, '清洗后': cleaned})
print(comparison)

---

## 处理重复数据

In [ ]:
#| eval: true
import pandas as pd

# 示例数据（含重复）
df = pd.DataFrame({
    '学号': ['001', '002', '002', '003', '003'],
    '姓名': ['张三', '李四', '李四', '王五', '王五'],
    '成绩': [85, 90, 90, 78, 80]  # 注意：002完全重复，003成绩不同
})

print("原始数据:")
print(df)

# 检查重复
print(f"\n完全重复的行数: {df.duplicated().sum()}")
print(f"学号重复的行数: {df.duplicated(subset=['学号']).sum()}")

# 删除完全重复的行
df_no_dup = df.drop_duplicates()
print("\n删除完全重复后:")
print(df_no_dup)

# 删除学号重复的行（保留第一个）
df_unique_id = df.drop_duplicates(subset=['学号'], keep='first')
print("\n删除重复学号后（保留第一个）:")
print(df_unique_id)

# 删除学号重复的行（保留最后一个）
df_unique_id_last = df.drop_duplicates(subset=['学号'], keep='last')
print("\n删除重复学号后（保留最后一个）:")
print(df_unique_id_last)

---

## 完整清洗流程

```{mermaid}
flowchart TD
    A["原始数据"] --> B["1. 查看数据概况"]
    B --> C["2. 处理重复值"]
    C --> D["3. 处理缺失值"]
    D --> E["4. 处理异常值"]
    E --> F["5. 统一格式"]
    F --> G["6. 验证数据质量"]
    G --> H["清洗后数据"]
```

### 综合案例

In [ ]:
#| eval: true
import pandas as pd
import numpy as np

def clean_student_data(df):
    """
    清洗学生数据的完整流程
    
    Returns:
        清洗后的 DataFrame 和清洗报告
    """
    report = {"原始行数": len(df)}
    df = df.copy()
    
    # 1. 删除完全重复的行
    df = df.drop_duplicates()
    report["删除完全重复后"] = len(df)
    
    # 2. 删除重复学号（保留第一条）
    df = df.drop_duplicates(subset=['学号'], keep='first')
    report["删除重复学号后"] = len(df)
    
    # 3. 处理成绩异常值：超出0-100的设为NaN
    if '成绩' in df.columns:
        invalid_mask = (df['成绩'] < 0) | (df['成绩'] > 100)
        report["异常成绩数"] = invalid_mask.sum()
        df.loc[invalid_mask, '成绩'] = np.nan
    
    # 4. 填充缺失的成绩（用均值）
    if '成绩' in df.columns:
        mean_score = df['成绩'].mean()
        df['成绩'] = df['成绩'].fillna(mean_score)
    
    # 5. 填充缺失的姓名
    if '姓名' in df.columns:
        df['姓名'] = df['姓名'].fillna('未知')
    
    report["最终行数"] = len(df)
    
    return df, report

# 测试
messy_data = pd.DataFrame({
    '学号': ['001', '002', '003', '003', '004', '005'],
    '姓名': ['张三', '李四', None, '王五', '赵六', '孙七'],
    '成绩': [85, 150, 78, 78, -5, None]
})

cleaned_df, report = clean_student_data(messy_data)

print("清洗报告:")
for key, value in report.items():
    print(f"  {key}: {value}")

print("\n清洗后数据:")
print(cleaned_df)

---

## 让 AI 帮你清洗数据

### Prompt 模板

```
我有一个学生成绩数据集，请帮我清洗：

数据问题：
1. 有些成绩是负数或超过100
2. 姓名有的是空值
3. 班级格式不统一（1班、一班、01班）
4. 有重复的学号

请生成 Pandas 清洗代码，并：
- 删除无法修复的异常值
- 填充缺失值
- 统一班级格式
- 报告清洗前后的数据量变化
```

---

## 课后作业

### 继续作业 4：数据可视化仪表板

**本周任务**：数据清洗

1. 下载真实数据集（包含质量问题）
2. 分析数据质量问题
3. 编写清洗代码
4. 生成清洗报告

**REPORT.md 要求**：
- 列出你发现的所有数据质量问题
- 展示清洗前后的对比

---

## 本章小结

- **数据质量问题**：缺失值、异常值、不一致、重复
- **缺失值处理**：删除、填充（均值/中位数），取决于缺失比例
- **异常值处理**：删除、截断、替换为 NaN
- **格式统一**：`pd.to_datetime()`、自定义清洗函数
- **重复处理**：`drop_duplicates()`
- **清洗流程**：先看概况 → 去重 → 处理缺失 → 处理异常 → 统一格式 → 验证

下一章，我们将学习**数据可视化**——用图表讲述数据故事。